# Obtaining an appropriate execution Environment 
## ie docker-stacks/neuronunit-optimization:

Force use of development local repository. The repository must be the russell_dev branch of https://github.com/russelljjarvis/neuronunit
and the docker build is: `docker-stacks/neuronunit-optimization`, there are more graceful ways of achieving this.

If you lack `docker-stacks/neuronunit-optimization` try:

```
$git clone -b dev https://github.com/scidash/docker-stacks.git
$cd docker-stacks
$sudo bash build-all```

The best way will be to integrate this doc chapter5 and its supporting code 
from https://github.com/russelljjarvis/neuronunit -> https://github.com/scidash/neuronunit

# Invocation of the jupyter Note book:

On my local machine the development repo of neuronunit is located here:

```
$HOME/git
# I navigate to that directory and execute:

docker run -p 8888:8888 -v \
`pwd`:/home/jovyan/mnt scidash/neuronunit-optimization \
jupyter notebook --ip=0.0.0.0 --NotebookApp.token=\"\" \ 
--NotebookApp.disable_check_xsrf=True 
```

In [1]:
import sys
import os
THIS_DIR = os.path.dirname(os.path.realpath('chapter5.ipynb'))
this_nu = os.path.join(THIS_DIR,'../')
sys.path.insert(0,this_nu)
import neuronunitdocker-stacks/neuronunit-optimization

In [ ]:
from neuronunit.tests import get_neab
from neuronunit.tests import utilities as outils                                    
from neuronunit.tests import model_parameters as modelp
import numpy as np
model = outils.model          
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 3
        gap.NGEN = 3
        # The population of genes is 12
        gap.MU = 12 
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]
        # call the actual Genetic Algorithm with Optimization parameters: number of generation (NGEN = 3)
        # and gene population size (MU = 12)
        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        attributes = [ i.attrs for i in vmpop ]
        rheobases = [ i.rheobase for i in vmpop ]
        scores = [ i.score for i in vmpop ]        
        parameters = [ i.attrs for i in vmpop ]
        
        data_tuples = (vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        return pop, data_tuples
        
    def _get_optimization_parameters(self, data_tuples):
        # Your specific unpacking of tuples that _optimize returns
        # vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        _ , parameters , scores, _ , _ , _ , _ = zip(*data_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        # The Class users version of optimize
        # where details are hidden in _optimizae
        
        # Inputs: 
        # a Izhihikitch model specified in NML, but implemented with a NEURONbackend type
        # from neuronunit. Modelp a formatated dictionary of model parameters
        # where keys are Izhikitich parameters, and values are parameter ranges.
        # Outputs:
        # the optimal model, the scores as pandas data frame.
        # data_tuples: other data about models from the converged gene population
        # like resulting rheobase values from the converged genes, pandas score arrays
        # the genes corresponding to attributes of the pareto front (in a raw format).
        '''

        # Do optimization including repeated calls to judge
        models, data_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(data_tuples)
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.
        # scores is a list of pandas dataframes for the converged gene population.
        # It might be good to convert it into one big panda table if I knew how.
        return model, scores, data_tuples

    
t = Test()
model,scores,data_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/docs/../neuronunit/tests/model_parameters.py'>
0 -50.8376340092
1 0.0790841869704
2 -56.5145353604
3 0.535614644172
4 -35.4335530981
5 37.2141844933
6 0.00138142948342
7 -63.3854869303
8 0.000104395426953
9 -4.55759441304e-09
0 -54.3704201198
1 0.0850739536203
2 -56.9739596038
3 0.118133085042
4 -32.1696559556
5 36.8356061609
6 0.00132337388855
7 -55.889148471
8 9.51439788988e-05
9 -3.90392712593e-09
0 -52.8552580956
1 0.17738222061
2 -58.5642829176
3 0.126300914176
4 -32.5570078597
5 39.5139091225
6 0.0010362079524
7 -57.2686984937
8 9.91522160196e-05
9 -4.02612586169e-09
0 -63.3265873323
1 0.0788477072512
2 -59.413786057
3 0.453040725825
4 -35.4164049051
5 39.2734027839
6 0.00110079301313
7 -50.0415748614
8 0.000103535575352
9 -3.66815278636e-09
0 -53.8872146395
1 0.0709786425056
2 -59.9403919311
3 0.944176660776
4 -39.9756516876
5 39.336238551
6 0.00110041235913
7 -52.7688673702
8 9.95798878271e-05
9 -3.57453894474e-

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


 -59.6528357604
3 0.0382789824488
4 -30.012199953
5 31.8230360623
6 0.000912806040142
7 -52.1637710458
8 9.52423241455e-05
9 -4.23675927932e-09
0 -56.4785129393
1 0.0571160740331
2 -59.3469861802
3 0.819687946392
4 -47.7704068002
5 39.4804010355
6 0.00110556242609
7 -61.0973527903
8 0.000108721514782
9 -3.54410605969e-09
0 -55.7289065055
1 0.0600293286857
2 -59.5202092386
3 0.293900036804
4 -36.7921318661
5 34.8038376854
6 0.00103202892685
7 -52.6503035154
8 0.000104513774633
9 -4.54229769983e-09
0 -70.2019412737
1 0.135322369073
2 -57.679120868
3 0.063590352026
4 -35.2991489553
5 30.5226934242
6 0.000778632095468
7 -71.640406687
8 0.00010020058141
9 -4.00819487068e-09
0 -69.6471786343
1 0.147888619027
2 -59.3961892114
3 0.781537115487
4 -49.1618887923
5 35.1773818744
6 0.00122983576646
7 -71.7925109634
8 9.85282093167e-05
9 -3.57578917646e-09
0 -62.8917825573
1 0.0833359456341
2 -58.442704232
3 0.0904068365789
4 -39.7346806831
5 31.0462743099
6 0.00125286256476
7 -54.4811480465
8 0.00

{}
<class 'sciunit.scores.ZScore'>


{}


...

<class 'float'>
157.63888888888889 vanilla {'injected_square_current': {'amplitude': '257.4074074074074', 'duration': '1000.0', 'delay': '100.0'}} (0, <neuronunit.tests.utilities.VirtualModel object at 0x7fef40405ac8>, 157.63888888888889)
{}
{}
{}


{}
<class 'sciunit.scores.ZScore'>


{}


{}


{}


In [ ]:
modelp
import pandas as pd
models = data_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0

In [ ]:
rheobases=data_tuples[5][0]

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0


In [ ]:
rheobases[0]


In [ ]:

sc1 = pd.DataFrame(scores[1])
sc1


In [ ]:
rheobases[1]


In [ ]:
data=[ models[1].name ]
model_values1 = pd.DataFrame(data)        
model_values1

In [ ]:
models[1].name        

The code below is used to get the differences between values obtained via brute force, and those obtained otherwise.
It displays the differences in parameter values as pandas data tables.
    
I have knowingly violated Github conventions by adding data (a pickled file, as well as sources to the repository). 
The justification being that ground_error (the ground truth to compare against Genetic Algorithm outputs). 
Takes a prohibitively long time to generate, and therefore detracts from notebooking philosophy.

In [ ]:
import pickle
import pandas as pd

try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    # The exception code is only skeletal, it would not actually work, but its the right principles.
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))

# ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
# pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = data_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])




In [ ]:
#These are the differences in attributes found via brute force versus the genetic algorithm. For the top two candidates.

df_0

In [ ]:
df_1
